In [1]:
!pip install transformers
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.6 MB/s eta 0:00:00


In [2]:
text = '2022 will be a great year for all of us'
encoding = tokenizer(text, add_special_tokens = True, truncation = True, padding ="max_length", return_attention_mask = True, return_tensors = "pt")
encoding['token_type_ids'].shape

torch.Size([1, 512])

BERT FOR MASK LANGUAGE MODELLING

In [3]:
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased', return_dict = True)
text = "The Opera House in Australia is in , "+ tokenizer.mask_token + " city"
input = tokenizer(text, return_tensors = "pt")
mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
output = model(**input)
logits = output.logits
softmax = F.softmax(logits, dim = -1)
print(mask_index)
mask_word = softmax[0, mask_index]
top_10_values, top_10_indices = torch.topk(mask_word, 10)

for token_index in top_10_indices[0]:
    word = tokenizer.decode([token_index])
    new_sentence = text.replace(tokenizer.mask_token, word)
    print(new_sentence)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(tensor([9]),)
The Opera House in Australia is in , sydney city
The Opera House in Australia is in , melbourne city
The Opera House in Australia is in , brisbane city
The Opera House in Australia is in , adelaide city
The Opera House in Australia is in , the city
The Opera House in Australia is in , canberra city
The Opera House in Australia is in , auckland city
The Opera House in Australia is in , hobart city
The Opera House in Australia is in , griffith city
The Opera House in Australia is in , hume city


BERT FOR NEXT SENTENCE PREDICTION

In [4]:
from transformers import BertForNextSentencePrediction
import torch
model1 = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
prompt = "I came back from Office in the evening"
next_sentence = "I opened my Beer after Office"
encoding = tokenizer(prompt, next_sentence, return_tensors='pt')
outputs = model1(**encoding)[0]
softmax = F.softmax(outputs, dim = 1)
print(softmax)

tensor([[9.9998e-01, 1.5085e-05]], grad_fn=<SoftmaxBackward0>)


BERT FOR QUESTION ANSWERING

In [5]:
text = "The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula.   The Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail.   In March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made available online.   The Vatican Secret Archives were separated from the library at the beginning of the 17th century; they contain another 150,000 items.   Scholars have traditionally divided the history of the library into five periods, Pre-Lateran, Lateran, Avignon, Pre-Vatican and Vatican.   The Pre-Lateran period, comprising the initial days of the library, dated from the earliest days of the Church. Only a handful of volumes survive from this period, though some are very significant."
question = "When was the Vat formally opened?"

In [6]:
from transformers import BertTokenizer, BertForQuestionAnswering
tokenizer = BertTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
model = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")
tokenized_inputs = tokenizer(question, text, return_tensors="pt")
with torch.no_grad():
        outputs = model(**tokenized_inputs)
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()
predict_answer_tokens = tokenized_inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


'1475'